## INTRODUCTION TO NLP
### Some differenc NLP Concepts/techniques


# Introduction


This is the assignment submission

The data set used comes from https://www.kaggle.com/datasets/crxxom/football-transfer-news-for-nlp

The dataset contains football transfer news from different sources. I only limited it to thelast month of the summer transfer window of 2023.

Its aim is just to explore how the different techniques work and how to perform them using different libraries.

Results for the first row are only going to be checked.

This notebook was created in and run on google colaboratory.


In [1]:
import pandas as pd
import numpy as np

import nltk
import spacy
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

# Download NLTK resources if not already downloaded
nltk.download('punkt')

# Initialize the stemmer
stemmer = PorterStemmer()

KeyboardInterrupt: 

In [3]:
#Mount google drive to be able to get the data set from there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Import the data set and copy it to a data frame
data = pd.read_csv("/content/drive/MyDrive/Unstructured Data Analytics/90minFootballTransferNewsNLP.csv")

df = data.copy()

In [6]:
df.head() #Get a feel of the data

,Title,Date,Link,Content
0,Football transfer rumours: Why Maguire's Man U...,"Aug 15, 2023",https://www.90min.com/posts/football-transfer-...,"90minÂ rounds up the latestÂ transfer news, ru..."
1,Chelsea agree Romeo Lavia fee with Southampton,"Aug 15, 2023",https://www.90min.com/posts/chelsea-agree-rome...,Chelsea have finalised an agreement with South...
2,Harry Maguire's proposed West Ham transfer col...,"Aug 15, 2023",https://www.90min.com/posts/harry-maguire-prop...,Harry Maguire's proposed transfer to West Ham ...
3,Southampton director breaks down Chelsea & Liv...,"Aug 15, 2023",https://www.90min.com/posts/southampton-direct...,Southampton director Jason Wilcox has revealed...
4,Neymar completes move from PSG to Al Hilal,"Aug 15, 2023",https://www.90min.com/posts/neymar-completes-m...,Saudi Pro League side Al Hilal have confirmed ...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6726 entries, 0 to 6725
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    6726 non-null   object
 1   Date     6726 non-null   object
 2   Link     6726 non-null   object
 3   Content  6726 non-null   object
dtypes: object(4)
memory usage: 210.3+ KB


### Some Cleaning of the data first

In [8]:
#Convert date to date time
df['Date'] = pd.to_datetime(df['Date'])
#Only get for the last month of the transfer window
df = df[(df['Date'].dt.month == 8) & (df['Date'].dt.year == 2023)]

In [9]:
df.drop(["Link","Date"], inplace=True, axis=1) #Drop columns which will not be used

<ipython-input-9-bf0b1f3cf572>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["Link","Date"], inplace=True, axis=1)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 1447
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    180 non-null    object
 1   Content  180 non-null    object
dtypes: object(2)
memory usage: 4.2+ KB


In [11]:
df.head()

,Title,Content
0,Football transfer rumours: Why Maguire's Man U...,"90minÂ rounds up the latestÂ transfer news, ru..."
1,Chelsea agree Romeo Lavia fee with Southampton,Chelsea have finalised an agreement with South...
2,Harry Maguire's proposed West Ham transfer col...,Harry Maguire's proposed transfer to West Ham ...
3,Southampton director breaks down Chelsea & Liv...,Southampton director Jason Wilcox has revealed...
4,Neymar completes move from PSG to Al Hilal,Saudi Pro League side Al Hilal have confirmed ...


From the above obeservation, after filtering we are down to 180 from 6726 rows and the data has only columns we are using

# Normalization

This is the process of transforming text into a standard or canonical form. This can involve several different steps like converting text to lowercase, removing punctuation, removing special characters, and converting numbers to their word representations. Normalization helps in reducing noise and ensuring consistency in text data, making it easier for further processing and analysis.

In [12]:

def normalize_sentence(sentence):
    # # Remove punctuation
    # sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # # Remove special characters and numbers
    # sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)

    # Replace special characters with their corresponding standard characters
    #This is beacse of the obeservation made on the data
    special_characters = {"Â": "A", "Ê": "E", "Û": "U", "â": "a", "ê": "e", "û": "u"}
    for char, replacement in special_characters.items():
        sentence = sentence.replace(char, replacement)

    # Convert text to lowercase and remove extra whitespaces
    sentence = ' '.join(sentence.lower().split())

    return sentence

def normalize_column(cell):
    # Tokenize the cell into sentences
    sentences = sent_tokenize(cell)

    # Normalize each sentence in the cell
    normalized_sentences = [normalize_sentence(sentence) for sentence in sentences]

    # Join the normalized sentences back into a single cell
    normalized_cell = ' '.join(normalized_sentences)

    return normalized_cell

In [13]:
df['normalized_Content'] = df['Content'].apply(normalize_column)

In [14]:
print(df["Content"].iloc[0])
print()
print(df["normalized_Content"].iloc[0])

90minÂ rounds up the latestÂ transfer news, rumours and gossip circulating around the world... West Ham have called off their move for Harry Maguire despite agreeing a Â£30m fee with Manchester United. The England defender wanted a severance payment of around Â£7m in order to depart Old Trafford and accept a lower wage packet at London Stadium. (Guardian) Manchester United would have needed to sign a replacement for Maguire had he chose to leave but they would have been left short of time to do so this window. (The Athletic) Despite reports suggesting otherwise, Everton have not received a bid from Manchester United for midfielder Amadou Onana, though would demand a huge transfer fee in order to sell the Belgian. (Liverpool Echo) Chelsea have agreed a Â£58m fee with Southampton for Romeo Lavia as Todd Boehly's spending during his time as owner nears the Â£1bn mark. (The Athletic) Manchester City will ramp up their interest in Jeremy Doku once they formalise a new bid for Lucas Paqueta.

##### Observation
From the above, its clearly visible that the normalized content is all in lowercase.All characters which are not in the basic english letter alphabet were also converted e.g.90minÂ to 90mina

# Stemming

This is an  informal process of converting words of their base forms using heuristic rules. Its primary goal is to reduce words to their base ro root form, known as the stem. It involves removing common prefixes and suffixes from words which results in a simplified form that represents the word’s core meaning.

In [15]:
import nltk
import spacy
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

# Download NLTK resources if not already downloaded
nltk.download('punkt')

# Initialize the stemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# Define a function to stem a single sentence
def stem_sentence(sentence):
    words = word_tokenize(sentence)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

# Define a function to stem each sentence in a column
def stem_column(column):
    sentences = sent_tokenize(column)
    stemmed_sentences = [stem_sentence(sentence) for sentence in sentences]
    return ' '.join(stemmed_sentences)

In [17]:
df['stemmed_Content'] = df['Content'].apply(stem_column)


In [18]:
print(df["Content"].iloc[0])
print()
print(df["stemmed_Content"].iloc[0])

90minÂ rounds up the latestÂ transfer news, rumours and gossip circulating around the world... West Ham have called off their move for Harry Maguire despite agreeing a Â£30m fee with Manchester United. The England defender wanted a severance payment of around Â£7m in order to depart Old Trafford and accept a lower wage packet at London Stadium. (Guardian) Manchester United would have needed to sign a replacement for Maguire had he chose to leave but they would have been left short of time to do so this window. (The Athletic) Despite reports suggesting otherwise, Everton have not received a bid from Manchester United for midfielder Amadou Onana, though would demand a huge transfer fee in order to sell the Belgian. (Liverpool Echo) Chelsea have agreed a Â£58m fee with Southampton for Romeo Lavia as Todd Boehly's spending during his time as owner nears the Â£1bn mark. (The Athletic) Manchester City will ramp up their interest in Jeremy Doku once they formalise a new bid for Lucas Paqueta.

###### Observation
When looking at the first two sentences, there is clearly some change where som words have been stemmed. E.g. The word rumours has been stemmed to rumour, circulating to circul. Although some words have been stemmed, some others have been stemmed which should not have been like the names of people which shows the downside of stemming.

# Lemmatization

This an advanced technique that uses vocabulary and morphological analysis to determine the base form of a word. It produces a valid base form that can be found in a dictionary, making it more accurate than stemming. It considers the context and converts the word to its meaningful base form, which is called Lemma

In [19]:
from nltk.stem import WordNetLemmatizer

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [20]:
# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Define a function to lemmatize a single sentence using spaCy
def lemmatize_sentence_spacy(sentence):
    doc = nlp(sentence)
    lemmatized_words = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_words)

# Define a function to lemmatize each sentence in a column using spaCy
def lemmatize_column(column):
    sentences = sent_tokenize(column)
    lemmatized_sentences = [lemmatize_sentence_spacy(sentence) for sentence in sentences]
    return ' '.join(lemmatized_sentences)

In [21]:
df['lemmatized_content'] = df['Content'].apply(lemmatize_column)

In [22]:
df.head()

,Title,Content,normalized_Content,stemmed_Content,lemmatized_content
0,Football transfer rumours: Why Maguire's Man U...,"90minÂ rounds up the latestÂ transfer news, ru...","90mina rounds up the latesta transfer news, ru...","90minâ round up the latestâ transfer news , ru...",90minÂ round up the latestâ transfer news ...
1,Chelsea agree Romeo Lavia fee with Southampton,Chelsea have finalised an agreement with South...,chelsea have finalised an agreement with south...,chelsea have finalis an agreement with southam...,Chelsea have finalise an agreement with Southa...
2,Harry Maguire's proposed West Ham transfer col...,Harry Maguire's proposed transfer to West Ham ...,harry maguire's proposed transfer to west ham ...,harri maguir 's propos transfer to west ham un...,Harry Maguire 's propose transfer to West Ham ...
3,Southampton director breaks down Chelsea & Liv...,Southampton director Jason Wilcox has revealed...,southampton director jason wilcox has revealed...,southampton director jason wilcox ha reveal he...,Southampton director Jason Wilcox have reveal ...
4,Neymar completes move from PSG to Al Hilal,Saudi Pro League side Al Hilal have confirmed ...,saudi pro league side al hilal have confirmed ...,saudi pro leagu side al hilal have confirm the...,Saudi Pro League side Al Hilal have confirm th...


In [23]:
print(df["Content"].iloc[0])
print()
print(df["lemmatized_content"].iloc[0])


90minÂ rounds up the latestÂ transfer news, rumours and gossip circulating around the world... West Ham have called off their move for Harry Maguire despite agreeing a Â£30m fee with Manchester United. The England defender wanted a severance payment of around Â£7m in order to depart Old Trafford and accept a lower wage packet at London Stadium. (Guardian) Manchester United would have needed to sign a replacement for Maguire had he chose to leave but they would have been left short of time to do so this window. (The Athletic) Despite reports suggesting otherwise, Everton have not received a bid from Manchester United for midfielder Amadou Onana, though would demand a huge transfer fee in order to sell the Belgian. (Liverpool Echo) Chelsea have agreed a Â£58m fee with Southampton for Romeo Lavia as Todd Boehly's spending during his time as owner nears the Â£1bn mark. (The Athletic) Manchester City will ramp up their interest in Jeremy Doku once they formalise a new bid for Lucas Paqueta.

# Text Enrichment / Augmentation (e.g.POS)

Text enrichment or augmentation involves adding additional information or annotations to text data. This can include part-of-speech (POS) tagging, which assigns grammatical categories to words in a sentence. POS tagging helps in understanding the syntactic structure of text and is useful for tasks like information extraction, parsing, and machine translation

In [24]:
# Define a function to enrich a single sentence with POS tags
def enrich_sentence(sentence):
    doc = nlp(sentence)
    enriched_sentence = {
        "tokens": [token.text for token in doc],
        "pos_tags": [token.pos_ for token in doc]
    }
    return enriched_sentence

# Define a function to enrich each sentence in a column
def enrich_column(column):
    sentences = column.split('. ')  # Assuming each sentence ends with a period followed by a space
    enriched_sentences = [enrich_sentence(sentence) for sentence in sentences]
    return enriched_sentences

In [25]:
df['enriched_text'] = df['Content'].apply(enrich_column)

In [26]:
print(df["Content"].iloc[0])
print()
print(df["enriched_text"].iloc[0])

90minÂ rounds up the latestÂ transfer news, rumours and gossip circulating around the world... West Ham have called off their move for Harry Maguire despite agreeing a Â£30m fee with Manchester United. The England defender wanted a severance payment of around Â£7m in order to depart Old Trafford and accept a lower wage packet at London Stadium. (Guardian) Manchester United would have needed to sign a replacement for Maguire had he chose to leave but they would have been left short of time to do so this window. (The Athletic) Despite reports suggesting otherwise, Everton have not received a bid from Manchester United for midfielder Amadou Onana, though would demand a huge transfer fee in order to sell the Belgian. (Liverpool Echo) Chelsea have agreed a Â£58m fee with Southampton for Romeo Lavia as Todd Boehly's spending during his time as owner nears the Â£1bn mark. (The Athletic) Manchester City will ramp up their interest in Jeremy Doku once they formalise a new bid for Lucas Paqueta.

###### Observation
From the above, its seen that there is  additional information or annotations to text data which was provided. We can see things like nouns, punctuations and other additional information added to the data.

# Named Entity Recognition

This is the task of identifying and classifying named entities in text into predefined categories such as person names, organization names, locations, dates, etc. Named Entity Recognition is crucial for extracting structured information from unstructured text data and is used in various applications like information retrieval, question answering, and entity linking.

In [27]:
# Define a function to perform named entity recognition on a single sentence
def ner_sentence(sentence):
    doc = nlp(sentence)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Define a function to perform named entity recognition on each sentence in a column
def ner_column(column):
    sentences = column.split('. ')  # Assuming each sentence ends with a period followed by a space
    ner_results = [ner_sentence(sentence) for sentence in sentences]
    return ner_results

In [28]:
df['ner_results'] = df['Content'].apply(ner_column)

In [29]:
print(df["Content"].iloc[0])
print()
print(df["ner_results"].iloc[0])

90minÂ rounds up the latestÂ transfer news, rumours and gossip circulating around the world... West Ham have called off their move for Harry Maguire despite agreeing a Â£30m fee with Manchester United. The England defender wanted a severance payment of around Â£7m in order to depart Old Trafford and accept a lower wage packet at London Stadium. (Guardian) Manchester United would have needed to sign a replacement for Maguire had he chose to leave but they would have been left short of time to do so this window. (The Athletic) Despite reports suggesting otherwise, Everton have not received a bid from Manchester United for midfielder Amadou Onana, though would demand a huge transfer fee in order to sell the Belgian. (Liverpool Echo) Chelsea have agreed a Â£58m fee with Southampton for Romeo Lavia as Todd Boehly's spending during his time as owner nears the Â£1bn mark. (The Athletic) Manchester City will ramp up their interest in Jeremy Doku once they formalise a new bid for Lucas Paqueta.

###### Observation
The entities in the text have been identified and classified, though not 100% accurate, majorly correctly. We can see Harry Maguire calssified as a person which is correct and Totenham also listed as person which is slightly innacurate.